This notebook documents how to evaluate the TCD model using k-fold cross validation. It assumes that

In [ ]:
from tcd_pipeline.modelrunner import ModelRunner
from tcd_pipeline.config import load_config
import os

In [ ]:
config = "../config/base_detectron.yaml"
base_config = load_config(config,  config_root=os.path.dirname(os.path.abspath(config)))

Make sure we have up-to-date weights:

gsutil -m cp gs://restor-deep-learning/models/segmentation/instance/kfold_0/20230315/model_final.pth ../checkpoints/kfold_0/model_final.pth
gsutil -m cp gs://restor-deep-learning/models/segmentation/instance/kfold_1/20230316/model_final.pth ../checkpoints/kfold_1/model_final.pth
gsutil -m cp gs://restor-deep-learning/models/segmentation/instance/kfold_2/20230316/model_final.pth ../checkpoints/kfold_2/model_final.pth
gsutil -m cp gs://restor-deep-learning/models/segmentation/instance/kfold_3/20230317/model_final.pth ../checkpoints/kfold_3/model_final.pth
gsutil -m cp gs://restor-deep-learning/models/segmentation/instance/kfold_4/20230318/model_final.pth ../checkpoints/kfold_4/model_final.pth

In [ ]:
folds = 5

In [ ]:
for i in range(folds):
    
    base_config['model']['weights'] = f"../checkpoints/kfold_{i}/model_final.pth"
    runner = ModelRunner(base_config)
    
    runner.evaluate(annotation_file=f"./kfold_{i}/test.json",
                    image_folder="../data/restor-tcd-oam/images/",
                    output_location=f"./kfold_{i}/eval/",
                    evaluate=True)

In [ ]:
def iou(bbox_a, bbox_b):
    
    x, y, w, h = bbox_a
    a = geometry.box(x, y, x+w, y+h)
    
    x, y, w, h = bbox_b
    b = geometry.box(x, y, x+w, y+h)
    
    intersect = a.intersection(b).area
    union = a.union(b).area
    iou = intersect / union
    
    return iou

def calculate_preds(gt_coco, eval_coco):
    preds = []
    labels = []
    check_empty_gt = []
    check_empty_pred = []

    iou_threshold = 0.2

    for idx in tqdm(gt_coco.imgs):
        # Load image at idx
        img = gt_coco.imgs[idx]
        gt_anns = gt_coco.imgToAnns[idx]

        # Skip if predictions don't exist
        try:
            with open(os.path.join(pred_root, os.path.splitext(img['file_name'])[0])+'.json') as fp:
                data = json.load(fp)
                dt_anns = data['predictions']
        except FileNotFoundError:
            continue

        # If both empty, probably correct
        if len(gt_anns) == 0 and len(dt_anns) == 0:
            continue
        # Check for confident detections with zero ground truth
        elif len(gt_anns) == 0 and len(dt_anns) > 0:
            check_empty_gt.append(img)
            continue
        # Check annotations for correctness as nothing was predicted
        elif len(gt_anns) > 0 and len(dt_anns) == 0:
            check_empty_pred.append(img)
            continue

        matches = np.zeros((len(gt_anns), len(dt_anns)))
        matches.shape

        for i, gt_ann in enumerate(gt_anns):
            for j, dt_ann in enumerate(dt_anns):
                matches[i, j] = iou(gt_ann['bbox'], dt_ann['bbox'])

        best_det_idx = np.argmax(matches, axis=1)

        for idx_gt, idx_dt in enumerate(best_det_idx):
            gt_ann = gt_anns[idx_gt]
            dt_ann = dt_anns[idx_dt]
            _iou = matches[idx_gt][idx_dt]

            if _iou > iou_threshold:
                labels.append(gt_ann)
                pred = dt_ann['probabilities']
                # TODO: FIX THIS URGENTLY FOR CONSISTENCY
                preds.append([pred[0], pred[2], pred[1]])
                ious.append(_iou)

In [ ]:
for i in range(folds):
    